In [1]:
%load_ext autoreload
%autoreload 2
import os 
os.chdir("/Users/luohy/Documents/Projects/bus-observatory/gtfs-realtime-performance")
from src.s3 import list_files_in_bucket, filter_files_by_pattern, read_parquet_from_s3
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.gtfs_segments import GTFS_shape_processor
from src.speeds import BusSpeedCalculator
from src.api import parse_zipped_gtfs
import geopandas as gpd
from src.api import query_feed_data, get_access_token
ACCESS_TOKEN = get_access_token()
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from collections import defaultdict
from shapely.geometry import Point, Polygon
from datetime import datetime
import pytz
import contextily as ctx


# Preparation 

In [2]:
prefix = "norm/bus-mta-vp/vehicles/"
bucket = "dataclinic-gtfs-rt"
mdb_id = "mdb-513"

In [3]:
def load_all_parquet_files(file_list, bucket, max_workers=100):
    dfs = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(read_parquet_from_s3, bucket, key) for key in file_list]
        for future in as_completed(futures):
            try:
                dfs.append(future.result())
            except Exception as e:
                print(f"Error reading a file: {e}")
    print(f"Read {len(dfs)} files from s3")
    return pd.concat(dfs, ignore_index=True)

In [4]:
def get_all_weekdays(start_date: str, end_date: str, weekday_type: int):
    # 0 - Monday, 1 - Tuesday, ..., 6 - Sunday
    # return a list of date strings
    dates = []
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    for date in date_range:
        if date.weekday() == weekday_type:
            dates.append(date.strftime('%Y-%m-%d'))
    return dates

## Step 1: Figure out which static feed is correct.

Pretty annoying. The trips and shapes are defined in the static gtfs feeds, and everytime a new static feed is released, the old `trip_id` values become obsolete. This is unfortunate because . It should be possible to just load all of the relevant static feeds and run a quick check on the relevant realtime feed data to see which static feed is in correct for the given dates. However, we can also just look through the outputs from MobilityData's API and find the one we want. Below, I load all the feeds for Manhattan.

TODO:
- save segment as geojson to /table-map

In [11]:
response = query_feed_data("mdb-513", ACCESS_TOKEN)
# https://mobilitydatabase.org/feeds/mdb-513
response

[{'id': 'mdb-513-202402080022',
  'feed_id': 'mdb-513',
  'hosted_url': 'https://files.mobilitydatabase.org/mdb-513/mdb-513-202402080022/mdb-513-202402080022.zip',
  'note': None,
  'downloaded_at': '2024-02-08T00:52:25.481924Z',
  'hash': '25ebc713eff01d5d5da7e6a877926d636152417153dc591ef0763fa0851b3819',
  'bounding_box': {'minimum_latitude': 40.701536,
   'maximum_latitude': 40.865316,
   'minimum_longitude': -74.018088,
   'maximum_longitude': -73.864375},
  'validation_report': None,
  'service_date_range_start': None,
  'service_date_range_end': None},
 {'id': 'mdb-513-202404010033',
  'feed_id': 'mdb-513',
  'hosted_url': 'https://files.mobilitydatabase.org/mdb-513/mdb-513-202404010033/mdb-513-202404010033.zip',
  'note': None,
  'downloaded_at': '2024-04-01T00:26:36.207481Z',
  'hash': '4e9fce8767c4fe3eb5f0529ea49d29270543d0563d7fbf0d8c4c0920b61a3fbf',
  'bounding_box': {'minimum_latitude': 40.701536,
   'maximum_latitude': 40.865316,
   'minimum_longitude': -74.018088,
   'max

In [10]:
def fetch_all_static_feeds(mdb_id: str, ACCESS_TOKEN: str) -> pd.DataFrame:
    response = query_feed_data(mdb_id, ACCESS_TOKEN)
    if response is None:
        raise ValueError("No response for mdb_id: ", mdb_id)
    # Convert the response to a DataFrame
    feed_updates = pd.DataFrame(response)
    # For each row where service_date_range_start is null, fill it with downloaded_at date
    if "service_date_range_start" in feed_updates.columns:
        extracted_dates = feed_updates["downloaded_at"].str.extract(r"(\d{4}-\d{2}-\d{2})")[0]
        feed_updates["service_date_range_start"] = feed_updates["service_date_range_start"].fillna(extracted_dates)  
    
    return feed_updates   

mdb_id = "mdb-513"
feed_updates = fetch_all_static_feeds(mdb_id, ACCESS_TOKEN=ACCESS_TOKEN)
preview_columns = ['service_date_range_start', 'hosted_url']
print(feed_updates[preview_columns])


  service_date_range_start                                         hosted_url
0               2024-02-08  https://files.mobilitydatabase.org/mdb-513/mdb...
1               2024-04-01  https://files.mobilitydatabase.org/mdb-513/mdb...
2               2024-07-01  https://files.mobilitydatabase.org/mdb-513/mdb...
3               2024-08-29  https://files.mobilitydatabase.org/mdb-513/mdb...
4               2024-09-09  https://files.mobilitydatabase.org/mdb-513/mdb...
5               2024-08-31  https://files.mobilitydatabase.org/mdb-513/mdb...
6               2025-01-04  https://files.mobilitydatabase.org/mdb-513/mdb...
7               2025-01-04  https://files.mobilitydatabase.org/mdb-513/mdb...
8               2025-02-09  https://files.mobilitydatabase.org/mdb-513/mdb...


In [14]:
# Select url and date range
url = "https://files.mobilitydatabase.org/mdb-513/mdb-513-202501020055/mdb-513-202501020055.zip"
start_date = "2025-01-02"
end_date = "2025-01-31"

# Based on the correct url
segment_df = GTFS_shape_processor(url, 4326, 2263).process_shapes()
GTFS_dict = parse_zipped_gtfs(url)

# Preview segment and GTFS_dict
print("--- segment_df.columns ---")
print(segment_df.columns)
print("--- GTFS_dict.keys() ---")
print(GTFS_dict.keys())

Parsed GTFS static feed data successfully, length:  8


/Users/luohy/Documents/Projects/bus-observatory/gtfs-realtime-performance/.venv/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


Parsed GTFS static feed data successfully, length:  8
--- segment_df.columns ---
Index(['trip_id', 'shape_id', 'stop_sequence', 'stop_id', 'stop_name',
       'prev_stop_id', 'prev_stop_name', 'projected_position',
       'prev_projected_position', 'segment_length', 'geometry'],
      dtype='object')
--- GTFS_dict.keys() ---
dict_keys(['agency.txt', 'calendar.txt', 'calendar_dates.txt', 'routes.txt', 'shapes.txt', 'stops.txt', 'stop_times.txt', 'trips.txt'])


# Step 2: Bus Speeds Calculation


In [29]:
def calculate_speeds_for_date_route(segment_df, GTFS_dict, bucket, date, route_id):

    # Load relevant realtime data from s3 bucket for the given date
    daily_files = list_files_in_bucket(bucket_name = bucket, prefix = f"{prefix}date={date}/")
    vehicle_positions = load_all_parquet_files(file_list = daily_files, bucket = bucket)
    # Filter by route
    vehicle_positions = vehicle_positions[vehicle_positions['trip.route_id'] == route_id]

    # Calculate speeds
    speed_calculator = BusSpeedCalculator(vehicle_positions, GTFS_dict, segment_df)
    speeds = speed_calculator.create_trip_speeds()
    speeds["route_id"] = route_id

    # Drop cols that are not needed
    speeds.drop(columns = ["stop_sequence", "stop_name", "prev_stop_name", "projected_position", "prev_projected_position", "unique_trip_id"], inplace = True)
    
    # Remove outlier
    speeds = speeds[speeds["speed_mph"] < 70]

    # Timezone conversion
    eastern_tz = pytz.timezone('America/New_York')
    speeds['interpolated_time'] = pd.to_datetime(speeds['interpolated_time'])
    speeds['datetime_nyc'] = speeds['interpolated_time'].dt.tz_localize('UTC').dt.tz_convert(eastern_tz)
    
    # add date column: datetime.date objects
    speeds["date"] = speeds["datetime_nyc"].dt.date
    # add weekday column: integer
    speeds["weekday"] = speeds["datetime_nyc"].dt.weekday
    # add hour column: integer
    speeds["hour"] = speeds["datetime_nyc"].dt.hour
    # drop interpolated_time
    speeds.drop(columns = ["interpolated_time"], inplace = True)

    return speeds

In [27]:
# Test 
# date = "2025-01-11"
# daily_files = list_files_in_bucket(bucket_name = bucket, prefix = f"{prefix}date={date}/")
# vehicle_positions = load_all_parquet_files(file_list = daily_files, bucket = bucket)

# print(vehicle_positions.columns)
# vehicle_positions.head()


Read 1405 files from s3
Index(['id', 'time', 'trip.trip_id', 'trip.route_id', 'trip.direction_id',
       'trip.start_time', 'trip.start_date', 'trip.schedule_relationship',
       'trip.modified_trip.modifications_id',
       'trip.modified_trip.affected_trip_id', 'vehicle.id', 'vehicle.label',
       'vehicle.license_plate', 'vehicle.wheelchair_accessible',
       'position.latitude', 'position.longitude', 'position.bearing',
       'position.odometer', 'position.speed', 'current_stop_sequence',
       'stop_id', 'current_status', 'timestamp', 'congestion_level',
       'occupancy_status', 'occupancy_percentage', 'multi_carriage_details.id',
       'multi_carriage_details.label',
       'multi_carriage_details.occupancy_status',
       'multi_carriage_details.occupancy_percentage',
       'multi_carriage_details.carriage_sequence', 'date'],
      dtype='object')


,id,time,trip.trip_id,trip.route_id,trip.direction_id,trip.start_time,trip.start_date,trip.schedule_relationship,trip.modified_trip.modifications_id,trip.modified_trip.affected_trip_id,vehicle.id,vehicle.label,vehicle.license_plate,vehicle.wheelchair_accessible,position.latitude,position.longitude,position.bearing,position.odometer,position.speed,current_stop_sequence,stop_id,current_status,timestamp,congestion_level,occupancy_status,occupancy_percentage,multi_carriage_details.id,multi_carriage_details.label,multi_carriage_details.occupancy_status,multi_carriage_details.occupancy_percentage,multi_carriage_details.carriage_sequence,date
0,MTA NYCT_8444,2025-01-11 05:07:58,JA_A5-Weekday-SDon-140000_MISC_912,Q85,0,,20250110,SCHEDULED,,,MTA NYCT_8444,,,NO_VALUE,40.661377,-73.721420,165.963760,0.0,0.0,0,503105,IN_TRANSIT_TO,1736572037,UNKNOWN_CONGESTION_LEVEL,EMPTY,0,None,None,None,None,None,2025-01-11
1,MTA NYCT_8447,2025-01-11 05:07:58,JA_A5-Weekday-SDon-142900_Q3_8,Q3,0,,20250110,SCHEDULED,,,MTA NYCT_8447,,,NO_VALUE,40.656494,-73.810265,294.863708,0.0,0.0,0,503804,IN_TRANSIT_TO,1736572032,UNKNOWN_CONGESTION_LEVEL,EMPTY,0,None,None,None,None,None,2025-01-11
2,MTA NYCT_9776,2025-01-11 05:07:58,MV_A5-Weekday-SDon-144500_M11_634,M104,1,,20250110,SCHEDULED,,,MTA NYCT_9776,,,NO_VALUE,40.813828,-73.956673,131.091461,0.0,0.0,0,403763,IN_TRANSIT_TO,1736572028,UNKNOWN_CONGESTION_LEVEL,EMPTY,0,None,None,None,None,None,2025-01-11
3,MTA NYCT_9769,2025-01-11 05:07:58,MV_A5-Weekday-SDon-142000_M3_343,M2,0,,20250110,SCHEDULED,,,MTA NYCT_9769,,,NO_VALUE,40.786236,-73.954880,54.162346,0.0,0.0,0,404191,IN_TRANSIT_TO,1736572044,UNKNOWN_CONGESTION_LEVEL,EMPTY,0,None,None,None,None,None,2025-01-11
4,MTA NYCT_9761,2025-01-11 05:07:58,MV_A5-Weekday-SDon-142500_M116_717,M10,0,,20250110,SCHEDULED,,,MTA NYCT_9761,,,NO_VALUE,40.800354,-73.958282,339.443970,0.0,0.0,0,401267,IN_TRANSIT_TO,1736572044,UNKNOWN_CONGESTION_LEVEL,EMPTY,0,None,None,None,None,None,2025-01-11


In [30]:
# # Test
# date = "2025-01-11"
# route_id = "M50"

# speeds = calculate_speeds_for_date_route(segment_df, GTFS_dict, bucket, date, route_id)

# speeds.head()

Read 1405 files from s3
Processing 78 trips...


100%|██████████| 78/78 [00:00<00:00, 405.63it/s]


,trip_id,shape_id,stop_id,prev_stop_id,segment_length,time_elapsed,speed_mph,route_id,datetime_nyc,date,weekday,hour
0,MQ_A5-Weekday-030500_M50_351,M500049,402185,405193,654.792033,44.0,10.146568,M50,2025-01-11 08:59:11-05:00,2025-01-11,5,8
1,MQ_A5-Weekday-030500_M50_351,M500049,402186,402185,536.962682,35.0,10.460309,M50,2025-01-11 08:59:46-05:00,2025-01-11,5,8
2,MQ_A5-Weekday-030500_M50_351,M500049,402188,402186,990.001139,61.0,11.065584,M50,2025-01-11 09:00:47-05:00,2025-01-11,5,9
3,MQ_A5-Weekday-030500_M50_351,M500049,402189,402188,555.668952,102.0,3.714364,M50,2025-01-11 09:02:29-05:00,2025-01-11,5,9
4,MQ_A5-Weekday-030500_M50_351,M500049,402191,402189,996.566792,81.0,8.388607,M50,2025-01-11 09:03:50-05:00,2025-01-11,5,9


In [33]:
# speeds.dtypes

trip_id                                     object
shape_id                                    object
stop_id                                      int64
prev_stop_id                                 int64
segment_length                             float64
time_elapsed                               float64
speed_mph                                  float64
route_id                                    object
datetime_nyc      datetime64[ns, America/New_York]
date                                        object
weekday                                      int32
hour                                         int32
dtype: object

# Step 3: batch - raw speeds
TODOs:
- nested loop: for dates, for route
    - can be improved with spark?

- design of parquet file
    - one large file?
    - partition by route? date?

# Step 4: batch - aggregation

TODOs:

- table for chart
- table for map